In [1]:
from mrrce import MrRCE
from simulation_utils import generate_data, me
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, RidgeCV, MultiTaskLassoCV
import matplotlib.pyplot as plt
%matplotlib inline

##### Generate Random Dataset

In [2]:
# settings
params = dict(
n = 50,                          # num obs
p = 20,                          # num predictors
q = 5,                           # num tasks
sigma = 1,                       # coeff variance
corr_x = .7,                     # grid of values for rho (correlation coefficient)
sigma_err = 1,                   # correlation coefficient for predictors
err_corr = .9,                   # correlation coefficient for errors
g_sparse_level = .0,             # group sparsity level
sparse_level = .0,               # sparsity level
err_cov_type = 'ar'             # error covariance form. One of ['ar', 'equi', 'fgn', 'identity']
)

In [3]:
np.random.seed(1)
rho = .4 # correlation coefficient for coefficients
X, Y, B, Sigma, Sigma_X = generate_data(rho = rho, **params)

##### Fit

In [4]:
m = MrRCE(max_iter=15, tol_glasso=.1, tol=1e-3, n_lams=10, verbose=True) # init MrRCE

In [5]:
m.fit(X, Y) # fit MrRCE

Iter number 7 

##### Estimations

In [6]:
m.B_hat[:1].round(5) # coefficient matrix

array([[-1.02498, -0.90094, -0.28147,  1.20656,  1.62085]])

In [7]:
B[:1].round(5)

array([[-0.9364 , -0.82716, -0.31097,  1.10047,  1.52656]])

In [8]:
print('True (sigma, rho) = ({sig}, {rho})\nEstimated (sigma, rho) = ({est_sig:.3f}, {est_rho:.3f})'.\
      format(sig = params['sigma'], rho = rho, est_sig = m.sigma, est_rho = m.rho))

True (sigma, rho) = (1, 0.4)
Estimated (sigma, rho) = (1.060, 0.418)


In [9]:
# transform Sigma
Sigma.round(5)

array([[ 0.28   ,  0.22217, -0.73113, -0.16708, -0.40396],
       [ 0.22217,  0.45187, -0.85959, -0.05453, -0.32333],
       [-0.73113, -0.85959,  3.0233 ,  0.49361,  1.41024],
       [-0.16708, -0.05453,  0.49361,  0.34337,  0.41362],
       [-0.40396, -0.32333,  1.41024,  0.41362,  0.90146]])

In [10]:
# estimated transformed Sigma
m.Sigma.round(5)

array([[ 0.12843,  0.0817 , -0.25172, -0.0625 , -0.12958],
       [ 0.0817 ,  0.32496, -0.52163, -0.02291, -0.12003],
       [-0.25172, -0.52163,  1.78125,  0.28318,  0.6322 ],
       [-0.0625 , -0.02291,  0.28318,  0.24294,  0.22089],
       [-0.12958, -0.12003,  0.6322 ,  0.22089,  0.43894]])

##### Compare to other estimators

In [11]:
# OLS
lm = LinearRegression(fit_intercept=False).fit(X, Y)
B_ols = np.matrix(lm.coef_.T)
# Ridge
ridge = RidgeCV(fit_intercept=False).fit(X, Y)
B_ridge = np.matrix(ridge.coef_.T)
# Group Lasso
gl = MultiTaskLassoCV(fit_intercept=False).fit(X, Y)
B_gl = np.matrix(gl.coef_.T)

In [12]:
# Model Error
print(
"""
Model Error:
============
MrRCE: {mrrce:.3f}
OLS: {ols:.3f}
GL: {gl:.3f}
Ridge: {ridge:.3f}
""".format(mrrce = me(B, m.B_hat, Sigma_X), # me for Model Error
           ols = me(B, B_ols, Sigma_X),
           gl = me(B, B_gl, Sigma_X),
           ridge = me(B, B_ridge, Sigma_X))
)


Model Error:
MrRCE: 1.809
OLS: 3.299
GL: 2.640
Ridge: 2.759

